In [10]:
import torch
import numpy as np
from torch import IntTensor
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, EarlyStoppingCallback


# DEVICE = 'cuda'
DEVICE = 'cpu'
MODEL_NAME = 'distilbert-base-uncased'
MAX_LENGTH = 512
TOKEN_SEP = 102
TOKEN_CLS = 101


model_name = MODEL_NAME
max_length = MAX_LENGTH

tokenizer = DistilBertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

sample_test = 'permissions on the target directory consider changing the remote tmp path in ansible configuration to a path rooted in tmp for more error information use v v v failed command was u mask'

# tokenizer.encode_plus(sample_test, padding=True, max_length=max_length, return_tensors='pt', truncation=True)
# dict_sample = tokenizer.encode_plus(sample_test, return_tensors='pt')
dict_sample = tokenizer.encode_plus(sample_test)

In [11]:
dict_sample

{'input_ids': [101, 6656, 2015, 2006, 1996, 4539, 14176, 5136, 5278, 1996, 6556, 1056, 8737, 4130, 1999, 2019, 19307, 9563, 2000, 1037, 4130, 15685, 1999, 1056, 8737, 2005, 2062, 7561, 2592, 2224, 1058, 1058, 1058, 3478, 3094, 2001, 1057, 7308, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
tokens_sample = list(set(dict_sample['input_ids'][1:-1]))

In [12]:
list_tokens_sample = [[token] for token in tokens_sample]
list_tokens_sample.insert(0, tokens_sample)

for tokens_sample in list_tokens_sample:
    tokens_sample.insert(0, TOKEN_CLS)
    tokens_sample.append(TOKEN_SEP)
# end

list_masks_attention = np.array([ [1 for i in range(len(tokens_sample))] for tokens_sample in list_tokens_sample])

<ipython-input-12-44e3f816c5a4>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  list_masks_attention = np.array([ [1 for i in range(len(tokens_sample))] for tokens_sample in list_tokens_sample])


In [13]:
list_info_input = [{'input_ids': torch.IntTensor([tokens_sample]), 'attention_mask': torch.IntTensor([masks_sample])} for tokens_sample, masks_sample in zip(list_tokens_sample, list_masks_attention)]

In [14]:
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=6)
model = model.to(DEVICE)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier

In [103]:
def prepare_input(sentence_origin, tokenizer):
    dict_sample = tokenizer.encode_plus(sentence_origin)
    
    tokens_sample = dict_sample['input_ids'][1:-1]
    tokens_sample_unique = list(set(dict_sample['input_ids'][1:-1]))

    list_tokens_sample = [[token] for token in tokens_sample_unique]
    list_tokens_sample.insert(0, tokens_sample)

    for tokens_sample in list_tokens_sample:
        tokens_sample.insert(0, TOKEN_CLS)
        tokens_sample.append(TOKEN_SEP)
    # end

    list_masks_attention = np.array([ [1 for i in range(len(tokens_sample))] for tokens_sample in list_tokens_sample])
    list_info_input = [{'input_ids': torch.IntTensor([tokens_sample]), 'attention_mask': torch.IntTensor([masks_sample]), 'is_main': False} for tokens_sample, masks_sample in zip(list_tokens_sample, list_masks_attention)]
    list_info_input[0]['is_main'] = True
    
    return list_info_input
# end

In [111]:
def parse_model_output(input_model, out_model, tokenizer):
    dict_info = {}
    
    tokens = input_model['input_ids'].tolist()[0]
    sentence = tokenizer.decode(tokens[1:-1])
    
    hidden_states = out_model.hidden_states
    h2 = hidden_states[-1].squeeze(0) # [24, 762]
    h2 = h2.to('cpu')
    
    is_main = input_model['is_main']
    
    if not is_main:
        # dict_info[sentence] = h2[0,:]   # get the cls of this token
        dict_info[sentence] = h2[1,:]   # get the token of this token
        return dict_info
    # end
    
    list_word = [tokenizer.decode(token) for token in tokens]
    list_result = []
    
    for i in range(len(list_word)):
        list_result.append([list_word[i], h2[i,:]])
    # end
    
    dict_info[sentence] = list_result
    
    return dict_info
# end


In [112]:
list_input = prepare_input(sample_test, tokenizer)
list_result = []

with torch.no_grad():
    for input_current in list_input:
        out = model(input_ids=input_current['input_ids'].to(DEVICE), attention_mask=input_current['attention_mask'].to(DEVICE), output_attentions=True, output_hidden_states=True)
        result = parse_model_output(input_current, out, tokenizer)
        list_result.append(result)
    # end for
# end

<ipython-input-103-3c9e8eb8d73d>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  list_masks_attention = np.array([ [1 for i in range(len(tokens_sample))] for tokens_sample in list_tokens_sample])


In [113]:
# hidden_states = out.hidden_states
# h2 = hidden_states[-1].squeeze(0) # [24, 762]
# h2 = h2.to('cpu')

In [114]:
# tokenizer.decode(list_info_input[0]['input_ids'].tolist()[0][1:-1])
# [tokenizer.decode(token) for token in list_info_input[0]['input_ids'].tolist()[0]]

In [115]:
# calculate token in sentence vs token short

dict_single = {k:v for result in list_result[1:] for k, v in result.items()}
list_origin = list(list_result[0].values())[0]
list_sim = []
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)

for item_origin in list_origin[1:-1]:
    embedding_origin = item_origin[1]
    embedding_single = dict_single[token_origin]
    
    result_cos = cos(embedding_origin, embedding_single).tolist()
    list_sim.append((token_origin, result_cos))
# end
    token_origin = item_origin[0]

In [119]:
# calculate token in sentence vs cls in sentence
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
list_sim = []

for item_origin in list_origin[1:-1]:
    token_origin = item_origin[0]
    
    embedding_origin = item_origin[1]
    embedding_cls = list_origin[0][1]

    result_cos = cos(embedding_origin, embedding_cls).tolist()
    list_sim.append((token_origin, result_cos))
# end

In [186]:
# [att for att in model.__dir__() if att[0] != '_' and callable(getattr(model, att))]

# target = model.distilbert.transformer.layer[-1].attention
# [att for att in target.__dir__() if att[0] != '_' and callable(getattr(target, att))]
# target.forward.__code__
# out.hidden_states[-1].shape

In [243]:
# from: https://github.com/huggingface/transformers/blob/main/src/transformers/models/distilbert/modeling_distilbert.py

from typing import Optional, Tuple
from types import MethodType
import math
from torch import nn

def forward_version_jinyuj(
    self,
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    mask: torch.Tensor,
    head_mask: Optional[torch.Tensor] = None,
    output_attentions: bool = False,
) -> Tuple[torch.Tensor, ...]:
    """
    Parameters:
        query: torch.tensor(bs, seq_length, dim)
        key: torch.tensor(bs, seq_length, dim)
        value: torch.tensor(bs, seq_length, dim)
        mask: torch.tensor(bs, seq_length)

    Returns:
        weights: torch.tensor(bs, n_heads, seq_length, seq_length) Attention weights context: torch.tensor(bs,
        seq_length, dim) Contextualized layer. Optional: only if `output_attentions=True`
    """
    
    print('[ jinyuj] run jinyuj version of forward')
    bs, q_length, dim = query.size()
    k_length = key.size(1)
    # assert dim == self.dim, f'Dimensions do not match: {dim} input vs {self.dim} configured'
    # assert key.size() == value.size()

    dim_per_head = self.dim // self.n_heads

    mask_reshp = (bs, 1, 1, k_length)

    def shape(x: torch.Tensor) -> torch.Tensor:
        """separate heads"""
        return x.view(bs, -1, self.n_heads, dim_per_head).transpose(1, 2)

    def unshape(x: torch.Tensor) -> torch.Tensor:
        """group heads"""
        return x.transpose(1, 2).contiguous().view(bs, -1, self.n_heads * dim_per_head)

    q = shape(self.q_lin(query))  # (bs, n_heads, q_length, dim_per_head)
    k = shape(self.k_lin(key))  # (bs, n_heads, k_length, dim_per_head)
    v = shape(self.v_lin(value))  # (bs, n_heads, k_length, dim_per_head)

    q = q / math.sqrt(dim_per_head)  # (bs, n_heads, q_length, dim_per_head)
    scores = torch.matmul(q, k.transpose(2, 3))  # (bs, n_heads, q_length, k_length)
    mask = (mask == 0).view(mask_reshp).expand_as(scores)  # (bs, n_heads, q_length, k_length)
    scores = scores.masked_fill(
        mask, torch.tensor(torch.finfo(scores.dtype).min)
    )  # (bs, n_heads, q_length, k_length)

    weights = nn.functional.softmax(scores, dim=-1)  # (bs, n_heads, q_length, k_length)
    weights = self.dropout(weights)  # (bs, n_heads, q_length, k_length)

    # Mask heads if we want to
    if head_mask is not None:
        weights = weights * head_mask

    context = torch.matmul(weights, v)  # (bs, n_heads, q_length, dim_per_head)
    context = unshape(context)  # (bs, q_length, dim)
    
    v_unshape = unshape(v)
    self.v_last = v_unshape
    self.z_last = context
    
    context = self.out_lin(context)  # (bs, q_length, dim)

    if output_attentions:
        return (context, weights)
    else:
        return (context,)

In [244]:
# model.distilbert.transformer.layer[-2].attention.forward.__code__
target = model.distilbert.transformer.layer[-1].attention
target.forward = MethodType(forward_version_jinyuj, target)

In [246]:
model(input_ids=input_current['input_ids'].to(DEVICE), attention_mask=input_current['attention_mask'].to(DEVICE), output_attentions=True, output_hidden_states=True)
print('done')

[ jinyuj] run jinyuj version of forward
done


In [248]:
target.z_last.shape

torch.Size([1, 3, 768])

In [237]:
# model.distilbert.transformer.layer[-1]
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       